# Week 3 Homework

ATTENTION: At the end of the submission form, you will be required to include a link to your GitHub repository or other public code-hosting site. This repository should contain your code for solving the homework. If your solution includes code that is not in file format (such as SQL queries or shell commands), please include these directly in the README file of your repository.

<b><u>Important Note:</b></u> <p> For this homework we will be using the 2022 Green Taxi Trip Record Parquet Files from the New York
City Taxi Data found here: </br> https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page </br>
If you are using orchestration such as Mage, Airflow or Prefect do not load the data into Big Query using the orchestrator.</br> 
Stop with loading the files into a bucket. </br></br>
<u>NOTE:</u> You will need to use the PARQUET option files when creating an External Table</br>

<b>SETUP:</b></br>
Create an external table using the Green Taxi Trip Records Data for 2022. </br>
Create a table in BQ using the Green Taxi Trip Records for 2022 (do not partition or cluster this table). </br>
</p>

### Observe URL patterns

- https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
- https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet
- https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-11.parquet
- https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-12.parquet

Same as for homework#2, can reuse. Now need to make that block a "common utils block" and pass the 
- base_URL
- colour
- year
- extension

or create separate functions for difference sources, as pattern remains the same in each source. Note Module-04 uses fhv data too

Needs to use chunking as shown in dlt workshop with generators as files are too big and too many files/

## Question 1: count of records

What is count of records for the 2022 Green Taxi Data??

### Answer 1: `840,402`

- 65,623,481
- 840,402
- 1,936,423
- 253,647

### Quick prototyping from local files

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
metadata = pq.read_metadata('../../../data/parquet/green_tripdata_2022-01.parquet')
print(metadata)

  created_by: parquet-cpp-arrow version 7.0.0
  num_columns: 20
  num_rows: 62495
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 10699


In [3]:
metadata.schema

required group field_id=-1 schema {
  optional int64 field_id=-1 VendorID;
  optional int64 field_id=-1 lpep_pickup_datetime (Timestamp(isAdjustedToUTC=false, timeUnit=microseconds, is_from_converted_type=false, force_set_converted_type=false));
  optional int64 field_id=-1 lpep_dropoff_datetime (Timestamp(isAdjustedToUTC=false, timeUnit=microseconds, is_from_converted_type=false, force_set_converted_type=false));
  optional binary field_id=-1 store_and_fwd_flag (String);
  optional double field_id=-1 RatecodeID;
  optional int64 field_id=-1 PULocationID;
  optional int64 field_id=-1 DOLocationID;
  optional double field_id=-1 passenger_count;
  optional double field_id=-1 trip_distance;
  optional double field_id=-1 fare_amount;
  optional double field_id=-1 extra;
  optional double field_id=-1 mta_tax;
  optional double field_id=-1 tip_amount;
  optional double field_id=-1 tolls_amount;
  optional int32 field_id=-1 ehail_fee (Null);
  optional double field_id=-1 improvement_surcharge

In [4]:
import io
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import fsspec
import duckdb

def load_data_from_api(*args, **kwargs):
    """

    """
    # setup the vars
    taxi_schema = pa.schema([
            ('VendorID', pa.int64()),
            ('lpep_pickup_datetime', pa.timestamp('us', tz='America/New_York')),
            ('lpep_dropoff_datetime', pa.timestamp('us', tz='America/New_York')),            
            ('passenger_count', pa.int64()),
            ('trip_distance', pa.float64()),
            ('RatecodeID', pa.int64()),
            ('store_and_fwd_flag', pa.string()),
            ('PULocationID', pa.int64()),
            ('DOLocationID', pa.int64()),
            ('payment_type', pa.int64()),
            ('fare_amount', pa.float64()),
            ('extra', pa.float64()),
            ('mta_tax', pa.float64()),
            ('tip_amount', pa.float64()),
            ('tolls_amount', pa.float64()),
            ('improvement_surcharge', pa.float64()),
            ('total_amount', pa.float64()),
            ('congestion_surcharge', pa.float64())
    ])

    months = range(1,13)
    year = 2022
    colour = 'green' # 'yellow'
    base_url="https://d37ci6vzurychx.cloudfront.net/trip-data"
    # dtc source for 2019-2021 does not have 2022 data 
    # ends at 2021-07 and only for csv.gz files, no parquet
    # base_url="https://github.com/DataTalksClub/nyc-tlc-data/releases/download"

    # Create empty list to store DataFrames
    dataframes = []

    # Connect to local DuckDB
    conn = duckdb.connect(database='ny_taxi_rides', read_only=False)

    # Iterate through months and download data
    for month in months:
        print(month)
        
        # local files
        filename = f"../../../data/parquet/{colour}_tripdata_{year}-{month:02d}.parquet" # .csv.gz / parquet
        # from nyc source
        # https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
        # filename = f"{colour}_tripdata_{year}-{month:02d}.parquet"
        # print(filename)

        # url = f"{base_url}/{filename}"
        # # print(url)

        # response = requests.get(url, stream=True)
        # if response.status_code == 200:
        #     fs = fsspec.filesystem("http")
        #     with fs.open(url) as f:
        #         table = pq.read_table(f, schema=taxi_schema)

        #         # Append DataFrame to the list
        #         dataframes.append(table.to_pandas())
        #         print(f"Downloaded {filename} successfully!")
        
        # else:
        #     print(f"Failed to download {filename}. Status code: {response.status_code}")

        table = pq.read_table(filename, schema=taxi_schema)

        # Append DataFrame to the list
        dataframes.append(table.to_pandas())
        print(f" {filename} appended successfully!")
        

    # # Concatenate DataFrames
    return pd.concat(dataframes, ignore_index=True)

df = load_data_from_api()
df


1
 ../../../data/parquet/green_tripdata_2022-01.parquet appended successfully!
2
 ../../../data/parquet/green_tripdata_2022-02.parquet appended successfully!
3
 ../../../data/parquet/green_tripdata_2022-03.parquet appended successfully!
4
 ../../../data/parquet/green_tripdata_2022-04.parquet appended successfully!
5
 ../../../data/parquet/green_tripdata_2022-05.parquet appended successfully!
6
 ../../../data/parquet/green_tripdata_2022-06.parquet appended successfully!
7
 ../../../data/parquet/green_tripdata_2022-07.parquet appended successfully!
8
 ../../../data/parquet/green_tripdata_2022-08.parquet appended successfully!
9
 ../../../data/parquet/green_tripdata_2022-09.parquet appended successfully!
10
 ../../../data/parquet/green_tripdata_2022-10.parquet appended successfully!
11
 ../../../data/parquet/green_tripdata_2022-11.parquet appended successfully!
12
 ../../../data/parquet/green_tripdata_2022-12.parquet appended successfully!


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2,2021-12-31 19:14:21-05:00,2021-12-31 19:15:33-05:00,1.0,0.44,1.0,N,42,42,2.0,3.50,0.50,0.5,0.00,0.0,0.3,4.80,0.00
1,1,2021-12-31 19:20:55-05:00,2021-12-31 19:29:38-05:00,1.0,2.10,1.0,N,116,41,2.0,9.50,0.50,0.5,0.00,0.0,0.3,10.80,0.00
2,1,2021-12-31 19:57:02-05:00,2021-12-31 20:13:14-05:00,1.0,3.70,1.0,N,41,140,1.0,14.50,3.25,0.5,4.60,0.0,0.3,23.15,2.75
3,2,2021-12-31 19:07:42-05:00,2021-12-31 19:15:57-05:00,1.0,1.69,1.0,N,181,181,2.0,8.00,0.50,0.5,0.00,0.0,0.3,9.30,0.00
4,2,2021-12-31 19:07:50-05:00,2021-12-31 19:28:52-05:00,1.0,6.26,1.0,N,33,170,1.0,22.00,0.50,0.5,5.21,0.0,0.3,31.26,2.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840397,2,2022-12-31 18:33:00-05:00,2022-12-31 18:49:00-05:00,NaN,3.53,NaN,None,33,79,NaN,20.72,0.00,0.0,4.89,0.0,1.0,29.36,NaN
840398,2,2022-12-31 18:29:00-05:00,2022-12-31 19:01:00-05:00,NaN,8.94,NaN,None,49,197,NaN,39.01,0.00,0.0,8.00,0.0,1.0,48.01,NaN
840399,2,2022-12-31 18:05:00-05:00,2022-12-31 18:12:00-05:00,NaN,1.27,NaN,None,54,181,NaN,12.11,0.00,0.0,2.62,0.0,1.0,15.73,NaN
840400,2,2022-12-31 18:03:00-05:00,2022-12-31 18:18:00-05:00,NaN,2.23,NaN,None,7,129,NaN,15.51,0.00,0.0,3.30,0.0,1.0,19.81,NaN


Code transferred to mage Data Loader block.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840402 entries, 0 to 840401
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype                           
---  ------                 --------------   -----                           
 0   VendorID               840402 non-null  int64                           
 1   lpep_pickup_datetime   840402 non-null  datetime64[us, America/New_York]
 2   lpep_dropoff_datetime  840402 non-null  datetime64[us, America/New_York]
 3   passenger_count        750062 non-null  float64                         
 4   trip_distance          840402 non-null  float64                         
 5   RatecodeID             750062 non-null  float64                         
 6   store_and_fwd_flag     750062 non-null  object                          
 7   PULocationID           840402 non-null  int64                           
 8   DOLocationID           840402 non-null  int64                           
 9   payment_type           750

In [6]:
pickup_years = df.lpep_pickup_datetime.dt.year
pickup_years.value_counts()

lpep_pickup_datetime
2022    840105
2021       290
2008         6
2009         1
Name: count, dtype: int64

In [7]:
dropoff_years = df.lpep_dropoff_datetime.dt.year
dropoff_years.value_counts()

lpep_dropoff_datetime
2022    840106
2021       284
2009         5
2023         5
2008         2
Name: count, dtype: int64

In [ ]:
-- import data from `mage-zoomcamp-ellacharmed.nyc_taxi_data.parquet`
-- Creating external table referring to gcs path
-- code snippet from 03-data-warehouse/big_query.sql
CREATE OR REPLACE EXTERNAL TABLE `nyc-rides-ella.ny_taxi.external_green_tripdata`
OPTIONS (
  format = 'PARQUET',
  uris = ['gs://mage-zoomcamp-ellacharmed/nyc_taxi_data.parquet']
);

In [ ]:
-- qn1 Create a non partitioned table from external table
CREATE OR REPLACE TABLE `nyc-rides-ella.ny_taxi.green_trips_2022_non_partitioned` AS
SELECT * 
FROM `nyc-rides-ella.ny_taxi.external_green_tripdata`;

In [ ]:
-- confirm row count is same from mage
SELECT COUNT(*) FROM `nyc-rides-ella.ny_taxi.external_green_tripdata`;

In [ ]:
-- confirm row count is same external table
SELECT COUNT(*) FROM `nyc-rides-ella.ny_taxi.green_trips_2022_non_partitioned`;

## Question 2: estimated data amount

### Answer 2: `0 MB for the External Table and 6.41MB for the Materialized Table`

Write a query to count the distinct number of PULocationIDs for the entire dataset on both the tables.</br> 
What is the estimated amount of data that will be read when this query is executed on the External Table and the Table?

- 0 MB for the External Table and 6.41MB for the Materialized Table
- 18.82 MB for the External Table and 47.60 MB for the Materialized Table
- 0 MB for the External Table and 0MB for the Materialized Table
- 2.14 MB for the External Table and 0MB for the Materialized Table

In [ ]:
-- qn 2, turn off cache results in query settins, select and view estimate
SELECT DISTINCT(PULocationID) 
FROM `nyc-rides-ella.ny_taxi.external_green_tripdata`;
-- shows 0B estimated, 6.41MB processed, 10MB billed

SELECT DISTINCT(PULocationID) 
FROM `nyc-rides-ella.ny_taxi.green_trips_2022_non_partitioned`;
-- shows 6.41MB estimated, 6.41MB processed, 10MB billed

## Question 3:
### Answer 3: `1,622`
How many records have a fare_amount of 0?
- 12,488
- 128,219
- 112
- 1,622

In [ ]:
-- qn 3 row count for trips with 0 fare_amount
SELECT COUNT(*)
FROM `nyc-rides-ella.ny_taxi.green_trips_2022_non_partitioned`
WHERE fare_amount=0;

## Question 4:
### Answer 4: `Partition by lpep_pickup_datetime  Cluster on PUlocationID`

What is the best strategy to make an optimized table in Big Query if your query will always order the results by PUlocationID and filter based on lpep_pickup_datetime? (Create a new table with this strategy)

- Cluster on lpep_pickup_datetime Partition by PUlocationID
- Partition by lpep_pickup_datetime  Cluster on PUlocationID
- Partition by lpep_pickup_datetime and Partition by PUlocationID
- Cluster on by lpep_pickup_datetime and Cluster on PUlocationID

In [ ]:
-- qn 4 create table with the strategy:
-- Partition by lpep_pickup_datetime  Cluster on PUlocationID
CREATE OR REPLACE TABLE `nyc-rides-ella.ny_taxi.green_trips_2022_optimized`
PARTITION BY DATE(lpep_pickup_dt)
CLUSTER BY PUlocationID AS
SELECT *, 
  TIMESTAMP_MICROS(CAST(lpep_pickup_datetime / 1000 AS INT64)) AS lpep_pickup_dt, 
  TIMESTAMP_MICROS(CAST(lpep_dropoff_datetime / 1000 AS INT64)) AS lpep_dropoff_dt 
FROM `nyc-rides-ella.ny_taxi.green_trips_2022_non_partitioned`;

## Question 5:
### Answer 5: `12.82 MB for non-partitioned table and 1.12 MB for the partitioned table`

Write a query to retrieve the distinct PULocationID between lpep_pickup_datetime
06/01/2022 and 06/30/2022 (inclusive)</br>

Use the materialized table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 4 and note the estimated bytes processed. What are these values? </br>

Choose the answer which most closely matches.</br> 

- 22.82 MB for non-partitioned table and 647.87 MB for the partitioned table
- 12.82 MB for non-partitioned table and 1.12 MB for the partitioned table
- 5.63 MB for non-partitioned table and 0 MB for the partitioned table
- 10.31 MB for non-partitioned table and 10.31 MB for the partitioned table

In [ ]:
SELECT UNIX_MICROS(TIMESTAMP "2022-06-01 00:00:00-05") AS start_date;
SELECT UNIX_MICROS(TIMESTAMP "2022-07-01 00:00:00-05")  AS end_date;

SELECT 
  DISTINCT(PULocationID)
  -- TIMESTAMP_TRUNC(lpep_pickup_datetime, DAY, "America/New_York") AS start_date,
  -- TIMESTAMP_TRUNC(lpep_pickup_datetime, DAY, "America/New_York") AS end_date
FROM 
  `nyc-rides-ella.ny_taxi.green_trips_2022_non_partitioned`
WHERE 
  lpep_pickup_datetime >= 1654059600000000
  and 
  lpep_pickup_datetime <= 1656651600000000
;

ToDo How to do the above in one SQL query?

In [ ]:
SELECT 
  DISTINCT(PULocationID)
  -- TIMESTAMP("2022-06-01 00:00:00", "America/New_York") AS start_dt,
  -- TIMESTAMP("2022-07-01 00:00:00", "America/New_York") AS end_dt
FROM `nyc-rides-ella.ny_taxi.green_trips_2022_optimized`
WHERE 
  lpep_pickup_dt >= TIMESTAMP("2022-06-01 00:00:00", "America/New_York") and 
  lpep_pickup_dt <= TIMESTAMP("2022-07-01 00:00:00", "America/New_York")
;

## Question 6: 
### Answer 6: `GCP Bucket`

Where is the data stored in the External Table you created?

- Big Query
- GCP Bucket
- Big Table
- Container Registry

## Question 7:
### Answer 7: `False`

It is best practice in Big Query to always cluster your data:
- True
- False

## (Bonus: Not worth points) Question 8:

No Points: Write a `SELECT count(*)` query FROM the materialized table you created. How many bytes does it estimate will be read? Why?

- BigQuery already has the answer so no extra processing is required
- the estimate may be due to caching?

## Submitting the solutions

* Form for submitting: https://courses.datatalks.club/de-zoomcamp-2024/homework/hw3
